In [227]:
%load_ext autoreload
%autoreload 2

import pyham
from pyoma.browser import db 
import numpy as np
from tables import *
import re
from ete3 import Tree
import pickle
import tempfile
import functools
import config

import pyhamPipeline
import profileGen
import format_files

from datasketch import MinHashLSH

import h5py

h5file = h5py.File(config.omadirLaurent + 'data1', 'a')

dataset_names = ['fam', 'duplication', 'gain', 'loss', 'presence']
list(h5file.keys())

for dataset_name in dataset_names:
    if dataset_name not in list(h5file.keys()):
        dataset = h5file.create_dataset(dataset_name, (0,0), maxshape=(None, None), dtype = 'int32')

dsets = {}
for dataset_name in list(h5file.keys()):
    dsets[dataset_name] = h5file[dataset_name]

#open up OMA
h5file = open_file(config.omadirLaurent + 'OmaServer.h5', mode="r") 
#setup db objects
dbObj = db.Database(h5file)
omaIdObj = db.OmaIdMapper(dbObj)

# corrects species tree and replacement dictionary for orthoXML files

dic, tree = format_files.create_species_tree(h5file, omaIdObj)
chunksize = 3

#temporary, 
mat_list = []
dico_list = []
lsh = MinHashLSH()

#load Fam
for i,fam in enumerate(pyhamPipeline.yieldFamilies(h5file)):
#generate pyham object
#     try: 
    
    if fam > 0 and fam < 5:
        print(fam)
        #generate treemap profile
        treemap_fam = pyhamPipeline.get_hamTree(fam, dbObj, tree, dic)
        # generate matrix of hash
        hashesDic = profileGen.Tree2Hashes(treemap_fam, fam, lsh)

        if i == 0:
            for dataset in dsets:
                if dataset == 'fam':
                    dsets[dataset].resize((chunksize, 1 ))
                else: 
                    dsets[dataset].resize((chunksize, np.asarray(hashesDic[dataset]).shape[0] ))
            
        if i % chunksize == 0 and i != 0:
            for dataset in dsets:
                if dataset == 'fam':
                    dsets[dataset].resize((i+chunksize, 1 ))
                else: 
                    dsets[dataset].resize((i+chunksize, np.asarray(hashesDic[dataset]).shape[0] ))


        for dset in dsets:
            if dset == 'fam':
                dsets[dset][i,:] = fam
            else:
                dsets[dset][i,:] = hashesDic[dset]
            
h5file.close()
#         dico_list.append(dico)
#         # generate taxa index
#         taxaIndex, taxaIndexReverse = profileGen.generateTaxaIndex(tree)

#         # generate matrix of 1 and 0 for each biological event
#         mat = profileGen.Tree2mat(treemap_fam, taxaIndex)
#         mat_list.append(mat)

#     except:
#         print("breaking at {}".format(fam))
        

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/home/laurent/anaconda3/lib/python3.6/site-packages/tables/leaf.py:396: PerformanceWarning: The Leaf ``/Protein/_i_Entries/OmaHOG/sorted`` is exceeding the maximum recommended rowsize (104857600 bytes);
be ready to see PyTables asking for *lots* of memory and possibly slow
I/O.  You may want to reduce the rowsize by trimming the value of
dimensions that are orthogonal (and preferably close) to the *main*
dimension of this leave.  Alternatively, in case you have specified a
very small/large chunksize, you may want to increase/decrease it.
  PerformanceWarning)


1
2
3
4
524
524
524
524


In [220]:
print(dsets['fam'].shape)

(6, 1)


In [229]:
h5file = h5py.File(config.omadirLaurent + 'data1', 'a')

h5file['gain']

<HDF5 dataset "gain": shape (6, 524), type "<i4">

In [230]:
h5file['gain'][1]

array([  1,   0,   0,   0,   0,   0,   0,   0, 128,   0,   0,   0, 232,
        76,  90,  55,  60, 150,   0, 156,  76, 234, 162, 215, 103, 137,
         0, 247, 154, 220, 202,  35, 212,  42, 222, 134, 148,  18, 213,
       126,  20,  99, 191, 102,  47, 138,  30, 159, 180,  82,  24, 183,
       154,   1,  67, 216, 115, 239, 159,  52,  86,   5, 118, 132, 176,
        39,  32,  99,  97, 134,  97, 124,  47, 179,  96,  93, 253,  22,
        71, 130, 171, 178,  22,   1, 221, 238, 235,   5,  28, 155, 117,
        12,  96, 196, 117, 174, 253, 198, 226, 105,  27,  19,  64, 218,
        89,  26, 235,  57,  24, 246, 248,  14, 201, 120, 137, 180, 178,
       255, 252, 113,  42,  45, 213, 113,  71, 204,  94,  36,  98, 244,
         9, 143,  60,  52, 216,  27,   7,  82,  67, 142,   2, 130, 151,
       183,  50,   9,  11,  40,  28,  22, 224,  92, 216,  15, 236, 252,
       252, 230, 163, 180,  55, 104, 203,  87,  33,  23, 143, 244, 245,
       196,  85, 157, 250, 127, 135, 230, 222,  95,  71,  57, 23

In [4]:
import h5py

f = h5py.File('hogTest.hdf5','w')
dset = f.create_dataset("test", (5,10), dtype=([("fam":"i"), ("duplication":"S128")]))


SyntaxError: invalid syntax (<ipython-input-4-9dd8862e7221>, line 4)

In [5]:
dset = f.create_dataset("test", (5,10), dtype([ ("field1": "i"), ("field2": "f") ]))



SyntaxError: invalid syntax (<ipython-input-5-61cdd2f5cda0>, line 1)